In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
=============================================================================
DUAL DATASET QUALITY CHECKER - I3D & RLT
=============================================================================
Comprehensive quality validation for both I3D and RLT datasets
Fully automated - processes both datasets sequentially
=============================================================================
Version: 2.0
Date: 2025-01-14
Author: Yeni Dwi Rahayu
Institution: Universitas Muhammadiyah Jember
=============================================================================
"""

import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# ==================== HELPER FUNCTIONS ====================
def convert_numpy_types(obj):
    """Convert NumPy types to Python native types for JSON serialization"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, tuple):
        return tuple(convert_numpy_types(item) for item in obj)
    elif pd.isna(obj):
        return None
    else:
        return obj


# ==================== CONFIGURATION ====================
class QualityCheckConfig:
    """Configuration for quality checks"""
    
    def __init__(self, dataset_name: str = 'I3D'):
        self.dataset_name = dataset_name
        self.base_dir = Path(os.getcwd())
        self.dataset_dir = self.base_dir / "dataset" / "processed" / dataset_name
        
        # Expected paths
        self.paths = {
            'text_indonesian': self.dataset_dir / "text" / "TextDataset_Indonesian.csv",
            'text_english': self.dataset_dir / "text" / "TextDataset_English.csv",
            'number_features': self.dataset_dir / "text" / "NumberFeatures.csv",
            'audio_features': self.dataset_dir / "audio" / "AudioDataset_Features.csv",
            'pause_features': self.dataset_dir / "audio" / "PauseFeatures.csv",
            'landmarks': self.dataset_dir / "visual" / "LandmarkDataset.csv",
            'multimodal_full': self.dataset_dir / "multimodal" / "MultimodalDataset_Full.csv",
            'publication': self.dataset_dir / "multimodal" / "PublicationDataset.csv"
        }
        
        # Output paths
        self.validation_dir = self.base_dir / "dataset" / "validation" / dataset_name
        self.quality_reports_dir = self.validation_dir / "quality_reports"
        self.quality_figures_dir = self.validation_dir / "quality_figures"
        
        # Create output directories
        self.quality_reports_dir.mkdir(parents=True, exist_ok=True)
        self.quality_figures_dir.mkdir(parents=True, exist_ok=True)
        
        # Quality thresholds
        self.thresholds = {
            'min_audio_quality': 0.3,
            'min_snr': 10,
            'min_duration': 0.5,
            'min_word_count': 3,
            'min_face_detection_rate': 50,  # %
            'min_iris_detection_rate': 30,  # %
            'max_missing_rate': 20,  # %
            'class_balance_tolerance': 0.2  # 40-60% is acceptable
        }


# ==================== FILE EXISTENCE CHECK ====================
def check_file_existence(config: QualityCheckConfig) -> Dict:
    """Check if all expected files exist"""
    
    print(f"\n{'='*70}")
    print(f"📁 CHECKING FILE EXISTENCE - {config.dataset_name}")
    print(f"{'='*70}")
    
    results = {
        'total_expected': len(config.paths),
        'found': 0,
        'missing': 0,
        'files': {}
    }
    
    for name, path in config.paths.items():
        exists = path.exists()
        
        if exists:
            size = path.stat().st_size
            size_mb = size / (1024 * 1024)
            results['found'] += 1
            results['files'][name] = {
                'exists': True,
                'path': str(path),
                'size_bytes': size,
                'size_mb': round(size_mb, 2)
            }
            print(f"✅ {name:25s} | {size_mb:8.2f} MB")
        else:
            results['missing'] += 1
            results['files'][name] = {
                'exists': False,
                'path': str(path)
            }
            print(f"❌ {name:25s} | MISSING")
    
    print(f"{'='*70}")
    print(f"📊 Summary: {results['found']}/{results['total_expected']} files found")
    
    if results['missing'] > 0:
        print(f"⚠️  WARNING: {results['missing']} files are missing!")
    else:
        print(f"✅ All expected files exist!")
    
    return results


# ==================== DATASET STRUCTURE CHECK ====================
def check_dataset_structure(config: QualityCheckConfig) -> Dict:
    """Check structure and basic statistics of each dataset"""
    
    print(f"\n{'='*70}")
    print(f"📊 CHECKING DATASET STRUCTURE - {config.dataset_name}")
    print(f"{'='*70}")
    
    results = {}
    
    for name, path in config.paths.items():
        if not path.exists():
            print(f"⏭️  Skipping {name} (file not found)")
            results[name] = {'status': 'missing'}
            continue
        
        try:
            df = pd.read_csv(path, encoding='utf-8')
            
            # Basic statistics
            n_rows = len(df)
            n_cols = len(df.columns)
            missing_total = df.isnull().sum().sum()
            missing_rate = (missing_total / (n_rows * n_cols) * 100) if n_rows * n_cols > 0 else 0
            
            # Check for label column
            label_col = None
            if 'label' in df.columns:
                label_col = 'label'
            elif 'Label' in df.columns:
                label_col = 'Label'
            elif 'Class' in df.columns:
                label_col = 'Class'
            
            has_label = label_col is not None
            
            if has_label:
                class_dist = df[label_col].value_counts().to_dict()
                n_truth = class_dist.get(0, 0)
                n_lie = class_dist.get(1, 0)
                balance = n_lie / (n_lie + n_truth) if (n_lie + n_truth) > 0 else 0
            else:
                class_dist = None
                n_truth = 0
                n_lie = 0
                balance = 0
            
            results[name] = {
                'status': 'ok',
                'n_rows': n_rows,
                'n_cols': n_cols,
                'missing_total': int(missing_total),
                'missing_rate': float(missing_rate),
                'has_label': has_label,
                'class_distribution': class_dist,
                'n_truth': int(n_truth),
                'n_lie': int(n_lie),
                'balance': float(balance),
                'columns': list(df.columns)
            }
            
            print(f"\n📄 {name}")
            print(f"   ├─ Rows: {n_rows:,}")
            print(f"   ├─ Columns: {n_cols}")
            print(f"   ├─ Missing values: {missing_total:,} ({missing_rate:.2f}%)")
            
            if has_label:
                print(f"   ├─ Class distribution:")
                print(f"   │  ├─ TRUTH (0): {n_truth}")
                print(f"   │  └─ LIE (1): {n_lie}")
                print(f"   └─ Class balance: {balance*100:.1f}% LIE")
                
                # Check balance
                if abs(balance - 0.5) > config.thresholds['class_balance_tolerance']:
                    print(f"      ⚠️  Class imbalance detected!")
            else:
                print(f"   └─ No label column found")
            
        except Exception as e:
            print(f"❌ Error reading {name}: {str(e)}")
            results[name] = {'status': 'error', 'error': str(e)}
    
    return results


# ==================== TEXT QUALITY CHECK ====================
def check_text_quality(config: QualityCheckConfig) -> Dict:
    """Check quality of text extraction"""
    
    print(f"\n{'='*70}")
    print(f"📝 CHECKING TEXT QUALITY - {config.dataset_name}")
    print(f"{'='*70}")
    
    results = {}
    
    # Check Indonesian text
    text_id_path = config.paths['text_indonesian']
    if text_id_path.exists():
        df_id = pd.read_csv(text_id_path)
        
        # Determine text column
        text_col = None
        if 'text_indonesian_normalized' in df_id.columns:
            text_col = 'text_indonesian_normalized'
        elif 'text_indonesian_original' in df_id.columns:
            text_col = 'text_indonesian_original'
        elif 'text_indonesian' in df_id.columns:
            text_col = 'text_indonesian'
        
        if text_col:
            empty_texts = df_id[text_col].isnull().sum() + (df_id[text_col] == '').sum()
            very_short = (df_id['word_count_id'] < config.thresholds['min_word_count']).sum() if 'word_count_id' in df_id.columns else 0
            
            results['indonesian'] = {
                'total_samples': len(df_id),
                'empty_texts': int(empty_texts),
                'very_short_texts': int(very_short),
                'avg_word_count': float(df_id['word_count_id'].mean()) if 'word_count_id' in df_id.columns else 0,
                'avg_char_count': float(df_id['char_count_id'].mean()) if 'char_count_id' in df_id.columns else 0,
                'avg_sentiment': float(df_id['sentiment_id'].mean()) if 'sentiment_id' in df_id.columns else 0,
                'avg_lexical_diversity': float(df_id['lexical_diversity_id'].mean()) if 'lexical_diversity_id' in df_id.columns else 0
            }
            
            print(f"\n📄 Indonesian Text:")
            print(f"   ├─ Total samples: {len(df_id)}")
            print(f"   ├─ Empty texts: {empty_texts} ({empty_texts/len(df_id)*100:.1f}%)")
            print(f"   ├─ Very short texts (< {config.thresholds['min_word_count']} words): {very_short}")
            print(f"   ├─ Avg word count: {results['indonesian']['avg_word_count']:.1f}")
            print(f"   ├─ Avg char count: {results['indonesian']['avg_char_count']:.1f}")
            print(f"   ├─ Avg sentiment: {results['indonesian']['avg_sentiment']:.3f}")
            print(f"   └─ Avg lexical diversity: {results['indonesian']['avg_lexical_diversity']:.3f}")
            
            if empty_texts > 0:
                print(f"      ⚠️  {empty_texts} samples have empty text!")
    
    # Check English text
    text_en_path = config.paths['text_english']
    if text_en_path.exists():
        df_en = pd.read_csv(text_en_path)
        
        if 'text_english' in df_en.columns:
            empty_texts = df_en['text_english'].isnull().sum() + (df_en['text_english'] == '').sum()
            very_short = (df_en['word_count_en'] < config.thresholds['min_word_count']).sum() if 'word_count_en' in df_en.columns else 0
            
            results['english'] = {
                'total_samples': len(df_en),
                'empty_texts': int(empty_texts),
                'very_short_texts': int(very_short),
                'avg_word_count': float(df_en['word_count_en'].mean()) if 'word_count_en' in df_en.columns else 0,
                'avg_char_count': float(df_en['char_count_en'].mean()) if 'char_count_en' in df_en.columns else 0,
                'translation_success_rate': float((len(df_en) - empty_texts) / len(df_en) * 100) if len(df_en) > 0 else 0
            }
            
            print(f"\n📄 English Text (Translation):")
            print(f"   ├─ Total samples: {len(df_en)}")
            print(f"   ├─ Empty translations: {empty_texts} ({empty_texts/len(df_en)*100:.1f}%)")
            print(f"   ├─ Translation success rate: {results['english']['translation_success_rate']:.1f}%")
            print(f"   ├─ Avg word count: {results['english']['avg_word_count']:.1f}")
            print(f"   └─ Avg char count: {results['english']['avg_char_count']:.1f}")
            
            if empty_texts > len(df_en) * 0.1:
                print(f"      ⚠️  More than 10% translations failed!")
    
    return results


# ==================== AUDIO QUALITY CHECK ====================
def check_audio_quality(config: QualityCheckConfig) -> Dict:
    """Check quality of audio features"""
    
    print(f"\n{'='*70}")
    print(f"🎵 CHECKING AUDIO QUALITY - {config.dataset_name}")
    print(f"{'='*70}")
    
    results = {}
    
    # Try to load from audio_features (which includes quality metrics)
    audio_path = config.paths['audio_features']
    if audio_path.exists():
        df = pd.read_csv(audio_path)
        
        # Check if quality columns exist
        has_quality = 'audio_quality_score' in df.columns
        
        if has_quality:
            # Quality metrics
            low_quality = (df['audio_quality_score'] < config.thresholds['min_audio_quality']).sum()
            low_snr = (df['audio_snr'] < config.thresholds['min_snr']).sum() if 'audio_snr' in df.columns else 0
            too_short = (df['audio_duration'] < config.thresholds['min_duration']).sum() if 'audio_duration' in df.columns else 0
            
            results = {
                'total_samples': len(df),
                'low_quality_count': int(low_quality),
                'low_quality_rate': float(low_quality / len(df) * 100) if len(df) > 0 else 0,
                'low_snr_count': int(low_snr),
                'too_short_count': int(too_short),
                'avg_quality_score': float(df['audio_quality_score'].mean()),
                'avg_snr': float(df['audio_snr'].mean()) if 'audio_snr' in df.columns else 0,
                'avg_duration': float(df['audio_duration'].mean()) if 'audio_duration' in df.columns else 0,
                'avg_rms': float(df['audio_rms'].mean()) if 'audio_rms' in df.columns else 0,
                'quality_distribution': {
                    'excellent': int((df['audio_quality_score'] >= 0.7).sum()),
                    'good': int(((df['audio_quality_score'] >= 0.5) & (df['audio_quality_score'] < 0.7)).sum()),
                    'fair': int(((df['audio_quality_score'] >= 0.3) & (df['audio_quality_score'] < 0.5)).sum()),
                    'poor': int((df['audio_quality_score'] < 0.3).sum())
                }
            }
            
            print(f"\n🎵 Audio Quality Metrics:")
            print(f"   ├─ Total samples: {len(df)}")
            print(f"   ├─ Avg quality score: {results['avg_quality_score']:.3f}")
            print(f"   ├─ Avg SNR: {results['avg_snr']:.1f} dB")
            print(f"   ├─ Avg duration: {results['avg_duration']:.2f} seconds")
            print(f"   ├─ Avg RMS: {results['avg_rms']:.4f}")
            print(f"   │")
            print(f"   ├─ Quality distribution:")
            print(f"   │  ├─ Excellent (≥0.7): {results['quality_distribution']['excellent']} ({results['quality_distribution']['excellent']/len(df)*100:.1f}%)")
            print(f"   │  ├─ Good (0.5-0.7): {results['quality_distribution']['good']} ({results['quality_distribution']['good']/len(df)*100:.1f}%)")
            print(f"   │  ├─ Fair (0.3-0.5): {results['quality_distribution']['fair']} ({results['quality_distribution']['fair']/len(df)*100:.1f}%)")
            print(f"   │  └─ Poor (<0.3): {results['quality_distribution']['poor']} ({results['quality_distribution']['poor']/len(df)*100:.1f}%)")
            print(f"   │")
            print(f"   ├─ Low quality samples (< {config.thresholds['min_audio_quality']}): {low_quality} ({results['low_quality_rate']:.1f}%)")
            print(f"   ├─ Low SNR samples (< {config.thresholds['min_snr']} dB): {low_snr}")
            print(f"   └─ Too short samples (< {config.thresholds['min_duration']}s): {too_short}")
            
            if results['low_quality_rate'] > 20:
                print(f"      ⚠️  More than 20% samples have low audio quality!")
        
        # Check pause features
        pause_path = config.paths['pause_features']
        if pause_path.exists():
            df_pause = pd.read_csv(pause_path)
            
            print(f"\n   📊 Pause Features: {len(df_pause.columns) - 3} features")  # -3 for filename, label, dataset
            
            if 'pause_hesitation_score' in df_pause.columns:
                avg_hesitation = df_pause['pause_hesitation_score'].mean()
                results['avg_hesitation_score'] = float(avg_hesitation)
                print(f"      ├─ Avg hesitation score: {avg_hesitation:.3f}")
            
            if 'pause_pause_frequency' in df_pause.columns:
                avg_pause_freq = df_pause['pause_pause_frequency'].mean()
                results['avg_pause_frequency'] = float(avg_pause_freq)
                print(f"      ├─ Avg pause frequency: {avg_pause_freq:.3f} pauses/sec")
            
            if 'pause_num_pauses' in df_pause.columns:
                avg_num_pauses = df_pause['pause_num_pauses'].mean()
                results['avg_num_pauses'] = float(avg_num_pauses)
                print(f"      └─ Avg number of pauses: {avg_num_pauses:.1f}")
    
    return results


# ==================== LANDMARK QUALITY CHECK ====================
def check_landmark_quality(config: QualityCheckConfig) -> Dict:
    """Check quality of landmark extraction"""
    
    print(f"\n{'='*70}")
    print(f"👤 CHECKING LANDMARK QUALITY - {config.dataset_name}")
    print(f"{'='*70}")
    
    results = {}
    
    landmark_path = config.paths['landmarks']
    if landmark_path.exists():
        print("📊 Loading landmark dataset (this may take a while)...")
        df = pd.read_csv(landmark_path)
        
        # Count unique videos
        unique_videos = df['Video_Name'].nunique() if 'Video_Name' in df.columns else 0
        total_frames = len(df)
        
        # Identify landmark columns
        face_cols = [col for col in df.columns if col.startswith('Landmark_') and 
                     col.endswith(('_X', '_Y', '_Z')) and 
                     int(col.split('_')[1]) < 468]
        
        iris_cols = [col for col in df.columns if col.startswith('Landmark_') and 
                     col.endswith(('_X', '_Y', '_Z')) and 
                     468 <= int(col.split('_')[1]) < 478]
        
        pose_cols = [col for col in df.columns if col.startswith('Pose_') and 
                     col.endswith(('_X', '_Y', '_Z'))]
        
        # Count frames with detected landmarks
        face_detected = (df[face_cols].sum(axis=1) != 0).sum() if face_cols else 0
        face_detection_rate = (face_detected / total_frames * 100) if total_frames > 0 else 0
        
        iris_detected = (df[iris_cols].sum(axis=1) != 0).sum() if iris_cols else 0
        iris_detection_rate = (iris_detected / total_frames * 100) if total_frames > 0 else 0
        
        pose_detected = (df[pose_cols].sum(axis=1) != 0).sum() if pose_cols else 0
        pose_detection_rate = (pose_detected / total_frames * 100) if total_frames > 0 else 0
        
        results = {
            'unique_videos': int(unique_videos),
            'total_frames': int(total_frames),
            'face_landmarks_count': len(face_cols) // 3,
            'iris_landmarks_count': len(iris_cols) // 3,
            'pose_landmarks_count': len(pose_cols) // 3,
            'face_detected': int(face_detected),
            'face_detection_rate': float(face_detection_rate),
            'iris_detected': int(iris_detected),
            'iris_detection_rate': float(iris_detection_rate),
            'pose_detected': int(pose_detected),
            'pose_detection_rate': float(pose_detection_rate)
        }
        
        print(f"\n👤 Landmark Extraction:")
        print(f"   ├─ Unique videos: {unique_videos}")
        print(f"   ├─ Total frames: {total_frames:,}")
        print(f"   │")
        print(f"   ├─ Face landmarks:")
        print(f"   │  ├─ Count: {results['face_landmarks_count']} landmarks")
        print(f"   │  ├─ Detected: {face_detected:,} frames")
        print(f"   │  └─ Detection rate: {face_detection_rate:.1f}%")
        print(f"   │")
        print(f"   ├─ Iris landmarks:")
        print(f"   │  ├─ Count: {results['iris_landmarks_count']} landmarks")
        print(f"   │  ├─ Detected: {iris_detected:,} frames")
        print(f"   │  └─ Detection rate: {iris_detection_rate:.1f}%")
        print(f"   │")
        print(f"   └─ Pose landmarks:")
        print(f"      ├─ Count: {results['pose_landmarks_count']} landmarks")
        print(f"      ├─ Detected: {pose_detected:,} frames")
        print(f"      └─ Detection rate: {pose_detection_rate:.1f}%")
        
        # Warnings
        if face_detection_rate < config.thresholds['min_face_detection_rate']:
            print(f"\n      ⚠️  Face detection rate is below {config.thresholds['min_face_detection_rate']}%!")
        
        if iris_detection_rate < config.thresholds['min_iris_detection_rate']:
            print(f"      ⚠️  Iris detection rate is below {config.thresholds['min_iris_detection_rate']}%!")
    
    return results


# ==================== CROSS-DATASET CONSISTENCY CHECK ====================
def check_cross_dataset_consistency(config: QualityCheckConfig) -> Dict:
    """Check consistency across different datasets"""
    
    print(f"\n{'='*70}")
    print(f"🔄 CHECKING CROSS-DATASET CONSISTENCY - {config.dataset_name}")
    print(f"{'='*70}")
    
    results = {}
    
    # Load all datasets
    datasets = {}
    
    for name, path in config.paths.items():
        if path.exists():
            try:
                datasets[name] = pd.read_csv(path)
            except:
                pass
    
    if not datasets:
        print("⚠️  No datasets loaded for consistency check")
        return results
    
    # Check sample counts
    sample_counts = {name: len(df) for name, df in datasets.items()}
    
    print(f"\n📊 Sample Counts:")
    for name, count in sample_counts.items():
        print(f"   ├─ {name:25s}: {count:,} samples")
    
    # Check if counts match (excluding landmarks which have frames)
    non_landmark_datasets = {k: v for k, v in sample_counts.items() if k != 'landmarks'}
    
    if non_landmark_datasets:
        counts = list(non_landmark_datasets.values())
        all_match = all(c == counts[0] for c in counts)
        
        results['sample_count_consistency'] = {
            'all_match': all_match,
            'counts': non_landmark_datasets
        }
        
        if all_match:
            print(f"\n   ✅ All non-landmark datasets have consistent sample counts: {counts[0]}")
        else:
            print(f"\n   ⚠️  Sample counts don't match across datasets!")
    
    # Check filename consistency
    if 'multimodal_full' in datasets and 'text_indonesian' in datasets:
        filenames_multi = set(datasets['multimodal_full']['filename'].values)
        filenames_text = set(datasets['text_indonesian']['filename'].values)
        
        missing_in_multi = filenames_text - filenames_multi
        missing_in_text = filenames_multi - filenames_text
        
        results['filename_consistency'] = {
            'match': len(missing_in_multi) == 0 and len(missing_in_text) == 0,
            'missing_in_multimodal': len(missing_in_multi),
            'missing_in_text': len(missing_in_text)
        }
        
        print(f"\n📄 Filename Consistency:")
        if results['filename_consistency']['match']:
            print(f"   ✅ All filenames match between datasets")
        else:
            print(f"   ⚠️  Filename mismatch detected:")
            print(f"      ├─ Missing in multimodal: {len(missing_in_multi)}")
            print(f"      └─ Missing in text: {len(missing_in_text)}")
    
    return results


# ==================== GENERATE QUALITY VISUALIZATIONS ====================
def generate_quality_visualizations(config: QualityCheckConfig):
    """Generate quality check visualizations"""
    
    print(f"\n{'='*70}")
    print(f"📊 GENERATING QUALITY VISUALIZATIONS - {config.dataset_name}")
    print(f"{'='*70}")
    
    sns.set_style("whitegrid")
    
    try:
        # Load datasets
        df_audio = pd.read_csv(config.paths['audio_features'])
        df_text_id = pd.read_csv(config.paths['text_indonesian'])
        
        # ===== VISUALIZATION 1: Audio Quality Distribution =====
        if 'audio_quality_score' in df_audio.columns:
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            
            # Quality score distribution
            axes[0, 0].hist(df_audio['audio_quality_score'], bins=30, color='#3498db', alpha=0.7, edgecolor='black')
            axes[0, 0].axvline(config.thresholds['min_audio_quality'], color='red', linestyle='--', 
                              label=f'Threshold ({config.thresholds["min_audio_quality"]})')
            axes[0, 0].set_xlabel('Audio Quality Score')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].set_title('Audio Quality Score Distribution')
            axes[0, 0].legend()
            axes[0, 0].grid(alpha=0.3)
            
            # SNR distribution
            if 'audio_snr' in df_audio.columns:
                axes[0, 1].hist(df_audio['audio_snr'], bins=30, color='#2ecc71', alpha=0.7, edgecolor='black')
                axes[0, 1].axvline(config.thresholds['min_snr'], color='red', linestyle='--', 
                                  label=f'Threshold ({config.thresholds["min_snr"]} dB)')
                axes[0, 1].set_xlabel('SNR (dB)')
                axes[0, 1].set_ylabel('Frequency')
                axes[0, 1].set_title('Signal-to-Noise Ratio Distribution')
                axes[0, 1].legend()
                axes[0, 1].grid(alpha=0.3)
            
            # Duration distribution
            if 'audio_duration' in df_audio.columns:
                axes[1, 0].hist(df_audio['audio_duration'], bins=30, color='#e74c3c', alpha=0.7, edgecolor='black')
                axes[1, 0].axvline(config.thresholds['min_duration'], color='red', linestyle='--', 
                                  label=f'Threshold ({config.thresholds["min_duration"]}s)')
                axes[1, 0].set_xlabel('Duration (seconds)')
                axes[1, 0].set_ylabel('Frequency')
                axes[1, 0].set_title('Audio Duration Distribution')
                axes[1, 0].legend()
                axes[1, 0].grid(alpha=0.3)
            
            # Quality categories
            quality_categories = pd.cut(df_audio['audio_quality_score'], 
                                        bins=[0, 0.3, 0.5, 0.7, 1.0],
                                        labels=['Poor', 'Fair', 'Good', 'Excellent'])
            quality_counts = quality_categories.value_counts()
            
            axes[1, 1].bar(quality_counts.index, quality_counts.values, 
                          color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'], 
                          alpha=0.7, edgecolor='black')
            axes[1, 1].set_xlabel('Quality Category')
            axes[1, 1].set_ylabel('Count')
            axes[1, 1].set_title('Audio Quality Categories')
            axes[1, 1].grid(alpha=0.3, axis='y')
            
            plt.suptitle(f'Audio Quality Analysis - {config.dataset_name}', fontsize=16, fontweight='bold')
            plt.tight_layout()
            plt.savefig(config.quality_figures_dir / 'audio_quality_analysis.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            print("✅ Audio quality visualization saved")
        
        # ===== VISUALIZATION 2: Text Quality Distribution =====
        if 'word_count_id' in df_text_id.columns:
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            
            # Word count distribution
            axes[0, 0].hist(df_text_id['word_count_id'], bins=30, color='#3498db', alpha=0.7, edgecolor='black')
            axes[0, 0].axvline(config.thresholds['min_word_count'], color='red', linestyle='--', 
                              label=f'Threshold ({config.thresholds["min_word_count"]})')
            axes[0, 0].set_xlabel('Word Count')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].set_title('Word Count Distribution (Indonesian)')
            axes[0, 0].legend()
            axes[0, 0].grid(alpha=0.3)
            
            # Sentiment distribution
            if 'sentiment_id' in df_text_id.columns:
                axes[0, 1].hist(df_text_id['sentiment_id'], bins=30, color='#2ecc71', alpha=0.7, edgecolor='black')
                axes[0, 1].set_xlabel('Sentiment Score')
                axes[0, 1].set_ylabel('Frequency')
                axes[0, 1].set_title('Sentiment Distribution')
                axes[0, 1].grid(alpha=0.3)
            
            # Lexical diversity
            if 'lexical_diversity_id' in df_text_id.columns:
                axes[1, 0].hist(df_text_id['lexical_diversity_id'], bins=30, color='#9b59b6', alpha=0.7, edgecolor='black')
                axes[1, 0].set_xlabel('Lexical Diversity')
                axes[1, 0].set_ylabel('Frequency')
                axes[1, 0].set_title('Lexical Diversity Distribution')
                axes[1, 0].grid(alpha=0.3)
            
            # Complexity
            if 'complexity_id' in df_text_id.columns:
                axes[1, 1].hist(df_text_id['complexity_id'], bins=30, color='#e74c3c', alpha=0.7, edgecolor='black')
                axes[1, 1].set_xlabel('Complexity Score')
                axes[1, 1].set_ylabel('Frequency')
                axes[1, 1].set_title('Text Complexity Distribution')
                axes[1, 1].grid(alpha=0.3)
            
            plt.suptitle(f'Text Quality Analysis - {config.dataset_name}', fontsize=16, fontweight='bold')
            plt.tight_layout()
            plt.savefig(config.quality_figures_dir / 'text_quality_analysis.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            print("✅ Text quality visualization saved")
        
    except Exception as e:
        print(f"❌ Error generating visualizations: {str(e)}")


# ==================== GENERATE QUALITY REPORT ====================
def generate_quality_report(all_results: Dict, config: QualityCheckConfig) -> Dict:
    """Generate comprehensive quality report"""
    
    print(f"\n{'='*70}")
    print(f"📝 GENERATING QUALITY REPORT - {config.dataset_name}")
    print(f"{'='*70}")
    
    report = {
        'report_info': {
            'generated_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'version': '2.0',
            'dataset_name': config.dataset_name,
            'dataset_directory': str(config.dataset_dir)
        },
        'file_existence': all_results.get('file_existence', {}),
        'dataset_structure': all_results.get('dataset_structure', {}),
        'text_quality': all_results.get('text_quality', {}),
        'audio_quality': all_results.get('audio_quality', {}),
        'landmark_quality': all_results.get('landmark_quality', {}),
        'cross_dataset_consistency': all_results.get('cross_dataset_consistency', {}),
        'thresholds_used': config.thresholds,
        'overall_assessment': {}
    }
    
    # Calculate overall assessment
    issues = []
    warnings = []
    
    # Check file existence
    if all_results.get('file_existence', {}).get('missing', 0) > 0:
        issues.append(f"{all_results['file_existence']['missing']} files are missing")
    
    # Check text quality
    text_quality = all_results.get('text_quality', {})
    if text_quality.get('indonesian', {}).get('empty_texts', 0) > 0:
        empty_count = text_quality['indonesian']['empty_texts']
        total = text_quality['indonesian']['total_samples']
        warnings.append(f"{empty_count} samples have empty text ({empty_count/total*100:.1f}%)")
    
    # Check audio quality
    audio_quality = all_results.get('audio_quality', {})
    if audio_quality.get('low_quality_rate', 0) > 20:
        warnings.append(f"High rate of low-quality audio: {audio_quality['low_quality_rate']:.1f}%")
    
    # Check landmark detection
    landmark_quality = all_results.get('landmark_quality', {})
    if landmark_quality.get('face_detection_rate', 0) < config.thresholds['min_face_detection_rate']:
        warnings.append(f"Low face detection rate: {landmark_quality['face_detection_rate']:.1f}%")
    
    if landmark_quality.get('iris_detection_rate', 0) < config.thresholds['min_iris_detection_rate']:
        warnings.append(f"Low iris detection rate: {landmark_quality['iris_detection_rate']:.1f}%")
    
    report['overall_assessment'] = {
        'status': 'PASS' if len(issues) == 0 else 'FAIL',
        'issues': issues,
        'warnings': warnings,
        'recommendation': 'Dataset is ready for use' if len(issues) == 0 and len(warnings) == 0 else 'Please review issues and warnings'
    }
    
    # Convert numpy types
    report = convert_numpy_types(report)
    
    # Save JSON report
    report_path = config.quality_reports_dir / f'quality_check_report_{config.dataset_name}.json'
    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)
    
    print(f"✅ JSON report saved: {report_path}")
    
    # Generate markdown summary
    markdown_summary = f"""# 🔍 Dataset Quality Check Report - {config.dataset_name}

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}  
**Dataset:** {config.dataset_name}  
**Directory:** `{config.dataset_dir}`

---

## 📊 Overall Assessment

**Status:** {'✅ PASS' if report['overall_assessment']['status'] == 'PASS' else '❌ FAIL'}

### Issues ({len(issues)})
{chr(10).join([f'- ❌ {issue}' for issue in issues]) if issues else '- ✅ No critical issues found'}

### Warnings ({len(warnings)})
{chr(10).join([f'- ⚠️ {warning}' for warning in warnings]) if warnings else '- ✅ No warnings'}

---

## 📁 File Existence

- **Total Expected:** {all_results.get('file_existence', {}).get('total_expected', 0)}
- **Found:** {all_results.get('file_existence', {}).get('found', 0)}
- **Missing:** {all_results.get('file_existence', {}).get('missing', 0)}

---

## 📊 Dataset Statistics

### Text Quality
- **Indonesian Total:** {text_quality.get('indonesian', {}).get('total_samples', 0)}
- **English Total:** {text_quality.get('english', {}).get('total_samples', 0)}
- **Avg Word Count (ID):** {text_quality.get('indonesian', {}).get('avg_word_count', 0):.1f}
- **Avg Word Count (EN):** {text_quality.get('english', {}).get('avg_word_count', 0):.1f}
- **Translation Success:** {text_quality.get('english', {}).get('translation_success_rate', 0):.1f}%

### Audio Quality
- **Total Samples:** {audio_quality.get('total_samples', 0)}
- **Avg Quality Score:** {audio_quality.get('avg_quality_score', 0):.3f}
- **Avg SNR:** {audio_quality.get('avg_snr', 0):.1f} dB
- **Low Quality Rate:** {audio_quality.get('low_quality_rate', 0):.1f}%

### Landmark Quality
- **Total Frames:** {landmark_quality.get('total_frames', 0):,}
- **Face Detection:** {landmark_quality.get('face_detection_rate', 0):.1f}%
- **Iris Detection:** {landmark_quality.get('iris_detection_rate', 0):.1f}%
- **Pose Detection:** {landmark_quality.get('pose_detection_rate', 0):.1f}%

---

## 💡 Recommendation

{report['overall_assessment']['recommendation']}

---

*Generated by Dual Dataset Quality Checker v2.0*
"""
    
    markdown_path = config.quality_reports_dir / f'QUALITY_CHECK_SUMMARY_{config.dataset_name}.md'
    with open(markdown_path, 'w', encoding='utf-8') as f:
        f.write(markdown_summary)
    
    print(f"✅ Markdown summary saved: {markdown_path}")
    
    return report


# ==================== DUAL PROCESSOR ====================
class DualQualityChecker:
    """Process both I3D and RLT datasets"""
    
    def __init__(self):
        self.datasets = ['I3D', 'RLT']
        self.results = {}
    
    def check_all(self):
        """Check both datasets"""
        print("="*70)
        print("🔍 DUAL DATASET QUALITY CHECKER")
        print("="*70)
        print("Checking quality for both I3D and RLT datasets")
        print("="*70)
        
        for dataset_name in self.datasets:
            print(f"\n\n{'#'*70}")
            print(f"# CHECKING: {dataset_name}")
            print(f"{'#'*70}")
            
            try:
                result = self.check_single_dataset(dataset_name)
                self.results[dataset_name] = result
            except Exception as e:
                print(f"\n❌ Error checking {dataset_name}: {str(e)}")
                import traceback
                traceback.print_exc()
                self.results[dataset_name] = {'status': 'error', 'message': str(e)}
        
        # Final summary
        self.print_final_summary()
    
    def check_single_dataset(self, dataset_name: str) -> Dict:
        """Check single dataset"""
        # Initialize
        config = QualityCheckConfig(dataset_name)
        
        # Store all results
        all_results = {}
        
        # Run checks
        all_results['file_existence'] = check_file_existence(config)
        all_results['dataset_structure'] = check_dataset_structure(config)
        all_results['text_quality'] = check_text_quality(config)
        all_results['audio_quality'] = check_audio_quality(config)
        all_results['landmark_quality'] = check_landmark_quality(config)
        all_results['cross_dataset_consistency'] = check_cross_dataset_consistency(config)
        
        # Generate visualizations
        generate_quality_visualizations(config)
        
        # Generate comprehensive report
        report = generate_quality_report(all_results, config)
        
        return {
            'status': 'success',
            'overall_status': report['overall_assessment']['status'],
            'issues_count': len(report['overall_assessment']['issues']),
            'warnings_count': len(report['overall_assessment']['warnings']),
            'reports_dir': str(config.quality_reports_dir)
        }
    
    def print_final_summary(self):
        """Print final summary for both datasets"""
        print("\n\n" + "="*70)
        print("📊 FINAL SUMMARY - BOTH DATASETS")
        print("="*70)
        
        for dataset_name, result in self.results.items():
            print(f"\n{dataset_name}:")
            
            if result.get('status') == 'success':
                print(f"  ✅ Status: {result.get('overall_status', 'UNKNOWN')}")
                print(f"  ├─ Issues: {result.get('issues_count', 0)}")
                print(f"  ├─ Warnings: {result.get('warnings_count', 0)}")
                print(f"  └─ Reports: {result.get('reports_dir', 'N/A')}")
            elif result.get('status') == 'error':
                print(f"  ❌ Status: ERROR")
                print(f"  └─ Message: {result.get('message', 'Unknown error')}")
        
        print("\n" + "="*70)
        print("✅ DUAL DATASET QUALITY CHECK COMPLETE!")
        print("="*70)


# ==================== MAIN ====================
def main():
    """Main execution"""
    checker = DualQualityChecker()
    checker.check_all()


if __name__ == "__main__":
    main()


🔍 DUAL DATASET QUALITY CHECKER
Checking quality for both I3D and RLT datasets


######################################################################
# CHECKING: I3D
######################################################################

📁 CHECKING FILE EXISTENCE - I3D
✅ text_indonesian           |     0.44 MB
✅ text_english              |     0.30 MB
✅ number_features           |     0.12 MB
✅ audio_features            |     2.91 MB
✅ pause_features            |     0.32 MB
✅ landmarks                 |  7536.75 MB
✅ multimodal_full           |     3.86 MB
✅ publication               |     0.71 MB
📊 Summary: 8/8 files found
✅ All expected files exist!

📊 CHECKING DATASET STRUCTURE - I3D

📄 text_indonesian
   ├─ Rows: 1,568
   ├─ Columns: 16
   ├─ Missing values: 0 (0.00%)
   ├─ Class distribution:
   │  ├─ TRUTH (0): 784
   │  └─ LIE (1): 784
   └─ Class balance: 50.0% LIE

📄 text_english
   ├─ Rows: 1,568
   ├─ Columns: 11
   ├─ Missing values: 0 (0.00%)
   ├─ Class distribution:
  